<a href="https://colab.research.google.com/github/SantiagoOrtegaR/Proyecto_Kaggle_Modelos/blob/main/02_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
#Instalar las librerias necesarias para la ejecución

!pip install gdown

import gdown
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [21]:
# ID del archivo en Google Drive
file_id = "1nWl7vNpI2XHd_B6cwAgIAYClQ3J9F6r1"
download_url = f"https://drive.google.com/uc?id={file_id}"

# Descargar el archivo CSV a local
output = "archivo.csv"
gdown.download(download_url, output, quiet=False)

# Leer CSV con pandas
df = pd.read_csv(output)


Downloading...
From (original): https://drive.google.com/uc?id=1nWl7vNpI2XHd_B6cwAgIAYClQ3J9F6r1
From (redirected): https://drive.google.com/uc?id=1nWl7vNpI2XHd_B6cwAgIAYClQ3J9F6r1&confirm=t&uuid=f6393cfb-e9ea-477a-af68-a3aa3c3ba3cf
To: /content/archivo.csv
100%|██████████| 144M/144M [00:01<00:00, 93.6MB/s]


In [22]:
# Valores faltantes
k = df.isna().sum()
k[k!=0]

,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENELAVADORA,39773
F_TIENEAUTOMOVIL,43623
E_PAGOMATRICULAPROPIO,6498
F_TIENECOMPUTADOR,38103
F_TIENEINTERNET.1,26629


In [23]:
#Limpieza de datos ordinales

orden_educacion = {
    'EducaciÃ³n profesional completa': 0,
    'EducaciÃ³n profesional incompleta': 1,
    'Ninguno': 2,
    'No Aplica': 3,
    'No sabe': 4,
    'Primaria completa': 5,
    'Postgrado': 6,
    'Primaria incompleta' : 7,
    'Secundaria (Bachillerato) completa' : 8,
    'Secundaria (Bachillerato) incompleta' : 9,
    'TÃ©cnica o tecnolÃ³gica completa' : 10,
    'TÃ©cnica o tecnolÃ³gica incompleta' : 11
}


orden_estrato = {
    'Sin estrato': 0,
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6
}

# Convertir a valores numéricos
df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].map(orden_educacion)
df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].map(orden_educacion)
df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].map(orden_estrato)

# Imputar con mediana
imp_median = SimpleImputer(strategy='median')
df[['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE', 'F_ESTRATOVIVIENDA']] = imp_median.fit_transform(
    df[['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE', 'F_ESTRATOVIVIENDA']]
)

# Volver a texto
reverse_educ = {v: k for k, v in orden_educacion.items()}
reverse_estrato = {v: k for k, v in orden_estrato.items()}

df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].astype(int).map(reverse_educ)
df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].astype(int).map(reverse_educ)
df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].astype(int).map(reverse_estrato)

# Columnas binarias (Si/No)
bin_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
             'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1']

imp_mode = SimpleImputer(strategy='most_frequent')
df[bin_cols] = imp_mode.fit_transform(df[bin_cols])

# Categóricas no ordinales
cat_cols = ['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA']
df[cat_cols] = imp_mode.fit_transform(df[cat_cols])

#df.to_excel("datos_estudiantes_limpios.xlsx", index=False)

In [24]:
# Chequeo valores faltantes
k = df.isna().sum()
k[k!=0]

,0


In [25]:
#Convertir columnas categoricas en binarias

cols_categoricas = ['F_TIENEINTERNET','F_TIENELAVADORA','F_TIENEAUTOMOVIL','E_PRIVADO_LIBERTAD','E_PAGOMATRICULAPROPIO','F_TIENECOMPUTADOR','F_TIENEINTERNET.1']
df[cols_categoricas] = df[cols_categoricas].apply(lambda x: x.str.strip().str.lower())
df[cols_categoricas] = df[cols_categoricas].replace({
    'si': 1, 's': 1,
    'no': 0, 'n': 0
})

# Columnas numéricas a normalizar
cols_numericas = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

scaler = MinMaxScaler(feature_range=(0, 1))
df[cols_numericas] = scaler.fit_transform(df[cols_numericas])

# Verificación rápida
print(df.head())

/tmp/ipython-input-2222237374.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[cols_categoricas] = df[cols_categoricas].replace({


       ID  PERIODO_ACADEMICO            E_PRGM_ACADEMICO E_PRGM_DEPARTAMENTO  \
0  904256              20212                  ENFERMERIA              BOGOTÁ   
1  645256              20212                     DERECHO           ATLANTICO   
2  308367              20203       MERCADEO Y PUBLICIDAD              BOGOTÁ   
3  470353              20195  ADMINISTRACION DE EMPRESAS           SANTANDER   
4  989032              20212                  PSICOLOGIA           ANTIOQUIA   

                E_VALORMATRICULAUNIVERSIDAD E_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones    Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                    0   
2  Entre 2.5 millones y menos de 4 millones      Más de 30 horas   
3  Entre 4 millones y menos de 5.5 millones                    0   
4  Entre 2.5 millones y menos de 4 millones  Entre 21 y 30 horas   

  F_ESTRATOVIVIENDA  F_TIENEINTERNET                    F_EDUCACIONPADRE  \
0         Estrato 3               

In [27]:
#Gurdar el nuevo df

df.to_csv('/content/drive/MyDrive/IA/train_clean.csv', index=False, encoding='utf-8-sig')